# Evaluación de PC-SMOTE con Grid Search en el dataset Shuttle (Generación de caso base y datasets aumentados)


In [1]:
# lo que hace es modificar la lista de rutas de búsqueda de módulos de Python (sys.path) para incluir las carpetas ../scripts y ../datasets como ubicaciones adicionales donde Python puede buscar módulos o paquetes cuando hacés un import.
import sys
sys.path.append("../scripts")
sys.path.append("../datasets")

## Importación de módulos y librerías necesarias


In [2]:
# --- Módulos propios del proyecto ---
from cargar_dataset import cargar_dataset                      # Función para cargar datasets según configuración
from config_datasets import config_datasets                    # Diccionario de configuración de datasets
from evaluacion import evaluar_sampler_holdout                 # Evaluación de sobremuestreo con partición hold-out
from custom_samplers import PCSMOTEWrapper                     # Wrapper personalizado para la técnica PCSMOTE
from pc_smote import PCSMOTE                                   # Implementación principal de PCSMOTE

# --- Librerías estándar de Python ---
from datetime import datetime, timedelta                       # Manejo de fechas y tiempos
from itertools import product                                  # Generación de combinaciones de parámetros
import gc, os, time                                                      # Operaciones con el sistema de archivos

# --- Librerías científicas ---
import numpy as np                                              # Operaciones numéricas y algebra lineal
import pandas as pd                                             # Manipulación y análisis de datos tabulares
from scipy.stats import uniform                                 # Distribuciones para búsqueda de hiperparámetros

# --- Scikit-learn: preprocesamiento ---
from sklearn.preprocessing import LabelEncoder, StandardScaler # Codificación de etiquetas y escalado de datos
from imblearn.pipeline import Pipeline as ImbPipeline            # Creación de pipelines de procesamiento y modelado

# --- Scikit-learn: división y validación ---
from sklearn.model_selection import (
    train_test_split,                                           # División de datos en train/test
    StratifiedKFold,                                            # Validación cruzada estratificada
    RandomizedSearchCV                                          # Búsqueda aleatoria de hiperparámetros
)

# --- Scikit-learn: reducción de dimensionalidad ---
from sklearn.decomposition import PCA                           # Análisis de Componentes Principales

# --- Scikit-learn: métricas ---
from sklearn.metrics import (
    f1_score,                                                    # Métrica F1-Score
    balanced_accuracy_score,                                     # Precisión balanceada
    matthews_corrcoef,                                           # Coeficiente MCC
    cohen_kappa_score,                                           # Kappa de Cohen
    make_scorer                                            
)

# --- Scikit-learn: clasificadores ---
from sklearn.ensemble import RandomForestClassifier             # Clasificador Random Forest
from sklearn.linear_model import LogisticRegression             # Regresión logística
from sklearn.svm import SVC                                      # Máquinas de Vectores de Soporte (SVM)
from scipy.stats import loguniform, randint, uniform

from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Evitar sobre-suscripción de CPU (BLAS/OpenMP)
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

'1'

### Evaluación de modelos con validación cruzada estratificada

Para evaluar el rendimiento de los modelos de clasificación sobre los datasets previamente balanceados, se utilizó validación cruzada estratificada de 5 particiones (Stratified K-Fold con *k=5*). Este método garantiza que en cada fold de entrenamiento y validación se preserve la proporción original de clases, lo cual es especialmente importante en tareas de clasificación multiclase con datasets balanceados artificialmente.

Durante el proceso, cada modelo es entrenado y evaluado cinco veces, cada vez usando un subconjunto distinto como conjunto de prueba y el resto como conjunto de entrenamiento. Las métricas calculadas en cada iteración (F1-score macro, balanced accuracy, MCC y kappa de Cohen) se promedian para obtener un valor representativo y del rendimiento general del modelo sobre ese dataset aumentado.

Este enfoque evita sobreajuste y proporciona una evaluación más confiable que una simple división train/test, permitiendo comparar de forma justa distintas configuraciones de sobremuestreo y modelos de clasificación.


In [3]:
import json, os

# --- Colocá esto una vez, arriba del script ---
def space_size(params: dict):
    """
    Devuelve el tamaño del espacio si es puramente discreto (listas/tuplas).
    Si detecta alguna distribución continua (tiene .rvs) o un valor no-discreto,
    retorna float('inf').
    """
    total = 1
    for v in params.values():
        if isinstance(v, (list, tuple)):
            total *= len(v)
        elif hasattr(v, "rvs"):  # scipy.stats
            return float("inf")
        else:
            return float("inf")
    return total


MAP_PATH = "../resultados/mapping_mejores_hparams_pipeline1.json"
with open(MAP_PATH, "r", encoding="utf-8") as f:
    MEJORES = json.load(f)

def get_clf_best_params(nombre_modelo: str, dataset: str, tipo: str):
    """
    Busca mejores hiperparámetros clf__* del pipeline 1.
    Si tipo == 'foldsafe', intenta 'base' (y luego 'aumentado').
    Orden de prueba:
      exacto -> 'base' -> 'aumentado' -> sin tipo
    """
    candidates = [
        str((nombre_modelo, dataset, tipo)),
        str((nombre_modelo, dataset, "base")) if tipo == "foldsafe" else None,
        str((nombre_modelo, dataset, "aumentado")) if tipo == "foldsafe" else None,
        str((nombre_modelo, dataset, "")),
    ]
    for k in filter(None, candidates):
        if k in MEJORES:
            return MEJORES[k]
    return {}


In [4]:
# -----------------------------
# Modelos base + espacios (afinados)
# -----------------------------
modelos = {
    "SVM": {
        "pipeline": ImbPipeline(steps=[
            ('scale', 'passthrough'),         # CSV ya escalados
            ('sampler', PCSMOTEWrapper()),    # aumento fold-safe dentro del CV
            ('clf', SVC(random_state=42, probability=False, max_iter=5000, cache_size=400))
        ]),
        "param_distributions": {
            "sampler__percentil_densidad": [25, 50, 75],
            "sampler__percentil_riesgo":   [25, 50, 75],
            "sampler__criterio_pureza":    ["entropia"],

            "clf__kernel": ['linear', 'rbf'],
            "clf__C":      loguniform(1e-3, 1e2),
            "clf__gamma":  loguniform(1e-4, 1e0),   # ignorado si kernel='linear'
            "clf__shrinking": [True, False],
            "clf__class_weight": [None, 'balanced']
        }
    },

    "LogisticRegression": {
        "pipeline": ImbPipeline(steps=[
            ('scale', 'passthrough'),         # CSV ya escalados
            ('sampler', PCSMOTEWrapper()),
            ('clf', LogisticRegression(max_iter=5000, n_jobs=1, random_state=42))
        ]),
        "param_distributions": {
            "sampler__percentil_densidad": [25, 50, 75],
            "sampler__percentil_riesgo":   [25, 50, 75],
            "sampler__criterio_pureza":    ["entropia"],

            "clf__C": loguniform(1e-3, 1e2),
            "clf__penalty": ["l2"],
            "clf__solver": ["lbfgs", "saga"],
            "clf__class_weight": [None, "balanced"]  # ← sugerido
        }
    },

    "RandomForest": {
        "pipeline": ImbPipeline(steps=[
            ('scale', 'passthrough'),         # RF no necesita escalado
            ('sampler', PCSMOTEWrapper()),
            ('clf', RandomForestClassifier(random_state=42))
        ]),
        "param_distributions": {
            "sampler__percentil_densidad": [25, 50, 75],
            "sampler__percentil_riesgo":   [25, 50, 75],
            "sampler__criterio_pureza":    ["entropia"],

            "clf__n_estimators": randint(150, 351),
            "clf__max_depth": [None, 5, 10, 15],
            "clf__max_features": ["sqrt", "log2"],
            "clf__min_samples_split": randint(2, 11),
            "clf__min_samples_leaf": randint(1, 6),
            "clf__bootstrap": [True],
            "clf__class_weight": [None, "balanced", "balanced_subsample"],
            "clf__ccp_alpha": uniform(0.0, 0.01)
        }
    }
}

# Orden: primero lo más rápido/estable
orden_modelos = ["LogisticRegression", "RandomForest", "SVM"]

# -----------------------------
# Rutas
# -----------------------------
ruta_base = "../datasets/datasets_aumentados/base/"
ruta_aug  = "../datasets/datasets_aumentados/pcsmote/"   # no se usa en fold-safe
dir_out  = "../resultados"   # no se usa en fold-safe

os.makedirs(dir_out, exist_ok=True)

# -----------------------------
# Listado de pares train/test
# -----------------------------
def listar_pares(ruta, tipo):
    pares = []
    if not os.path.isdir(ruta):
        return pares
    for f in os.listdir(ruta):
        if not (f.endswith("_train.csv") and os.path.isfile(os.path.join(ruta, f))):
            continue
        path_train = os.path.join(ruta, f)
        path_test  = os.path.join(ruta, f.replace("_train.csv", "_test.csv"))
        if not os.path.isfile(path_test):
            print(f"⚠️ Falta el test para: {f} -> esperado: {os.path.basename(path_test)}")
            continue
        pares.append({
            "tipo": tipo,
            "nombre_train": f,
            "path_train": path_train,
            "path_test": path_test
        })
    return pares

# ✅ Fold-safe: solo base (sin CSVs aumentados)
pares = listar_pares(ruta_base, "foldsafe")
pares = sorted(pares, key=lambda x: (x["tipo"], x["nombre_train"]))
total_pares = len(pares)

# -----------------------------
# Métricas CV
# -----------------------------
scoring = {
    'f1_macro': 'f1_macro',
    'balanced_accuracy': 'balanced_accuracy',
    'mcc': make_scorer(matthews_corrcoef),
    'cohen_kappa': make_scorer(cohen_kappa_score)
}

# -----------------------------
# Resumible: claves ya calculadas
# -----------------------------
def cargar_claves_existentes():
    done = {m: set() for m in modelos}
    for m in modelos:
        p = os.path.join(dir_out, f"resultados_{m}.csv")
        if os.path.exists(p):
            try:
                df_prev = pd.read_csv(p)
                for _, r in df_prev.iterrows():
                    done[m].add((
                        str(r.get('dataset')), str(r.get('tipo', '')),
                        str(r.get('tecnica')), str(r.get('densidad')),
                        str(r.get('riesgo')), str(r.get('pureza'))
                    ))
            except Exception:
                pass
    return done

ya_hechos = cargar_claves_existentes()
resultados_por_modelo = {nombre: [] for nombre in modelos}

# -----------------------------
# Helper de carga (float32)
# -----------------------------
def cargar_xy(path_csv):
    df = pd.read_csv(path_csv)
    if "target" in df.columns:
        X = df.drop(columns=["target"]).to_numpy(dtype=np.float32, copy=False)
        y = df["target"].to_numpy()
    else:
        X = df.iloc[:, :-1].to_numpy(dtype=np.float32, copy=False)
        y = df.iloc[:, -1].to_numpy()
    return X, y

# =============================
# Loop principal
# =============================
for idx_par, item in enumerate(pares, start=1):
    archivo_train = item["nombre_train"]
    ruta_train = item["path_train"]; ruta_test = item["path_test"]
    tipo = item["tipo"]  # 'foldsafe'

    # Parseo nombre (solo base fold-safe)
    nombre = archivo_train.replace(".csv", "")
    nombre_dataset = nombre.replace("_train", "")
    tecnica = "pcsmote"                 # sampler dentro del CV
    densidad = riesgo = pureza = "NA"   # se completan con best_params

    print(f"\n📂 ({idx_par}/{total_pares}) Par: {archivo_train}  (tipo: {tipo})")
    print(f"🔎 Técnica: {tecnica} | Dataset: {nombre_dataset}")


    # Carga de datos
    try:
        X_train, y_train = cargar_xy(ruta_train)
        X_test,  y_test  = cargar_xy(ruta_test)
    except Exception as e:
        print(f"❌ Error al leer train/test ({archivo_train}): {e}")
        continue

    n_samples, n_features = X_train.shape
    es_grande  = (n_samples >= 10000) or (nombre_dataset.lower() == "shuttle")
    es_shuttle = (nombre_dataset.lower() == "shuttle")

    # CV / iteraciones / paralelismo
    cv_base    = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_grande  = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    cv_shuttle = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

    if es_shuttle:
        cv_actual = cv_shuttle
        n_iter_default = 5  # liviano
    else:
        cv_actual = (cv_grande if es_grande else cv_base)
        n_iter_default = (5 if es_grande else 10)

    cpu = os.cpu_count() or 4
    n_jobs_default = 1 if es_grande else max(1, min(4, cpu // 2))

    # -------------------------
    # Por modelo (ordenado)
    # -------------------------
    for nombre_modelo in orden_modelos:
        info = modelos[nombre_modelo]

        # Evitar duplicados si ya existe salida previa con la misma clave base
        key_base = (nombre_dataset, 'foldsafe', tecnica, str(densidad), str(riesgo), str(pureza))
        if key_base in ya_hechos[nombre_modelo]:
            print(f"⏭️  {nombre_modelo}: ya existe {key_base}; se omite (clave base).")

        # Shuttle → evitar SVM por tiempo
        if es_shuttle and nombre_modelo == "SVM":
            print("⏭️  SVM omitido para Shuttle (regla de rendimiento).")
            continue

        print(f"⚙️ Validando modelo con RandomizedSearchCV: {nombre_modelo}")

        # Config local + espacio adaptado (partimos de lo definido en `modelos`)
        pipe = info["pipeline"]
        param_distributions = info["param_distributions"].copy()

        # Ajustes por dataset grande / shuttle
        if nombre_modelo == "SVM" and es_grande:
            param_distributions["clf__kernel"] = ['linear']
            param_distributions["clf__C"] = uniform(0.1, 10.0)
            param_distributions["clf__gamma"] = ['scale']  # ignorado con 'linear'

        if nombre_modelo == "LogisticRegression" and es_shuttle:
            param_distributions.update({
                "clf__solver": ["saga"],
                "clf__penalty": ["l2"],
                "clf__C": loguniform(1e-3, 1e1),
                "clf__class_weight": ["balanced"]
            })

        if nombre_modelo == "RandomForest" and es_shuttle:
            param_distributions["clf__max_samples"] = uniform(0.5, 0.45)  # 0.5–0.95

        n_iter_actual = n_iter_default
        n_jobs_actual = n_jobs_default

        # Búsqueda
        try:

            # clf_best = get_clf_best_params(nombre_modelo, nombre_dataset, tipo)
            # if clf_best:
            #     pipe.set_params(**clf_best)
            #     # eliminar cualquier búsqueda sobre clf__*
            #     param_distributions = {k:v for k,v in param_distributions.items()
            #                         if not (k.startswith("clf__") or k.startswith("classifier__"))}
            #     if not param_distributions:
            #         param_distributions = {
            #             "sampler__percentil_densidad": [25, 50, 75],
            #             "sampler__percentil_riesgo":   [25, 50, 75],
            #             "sampler__criterio_pureza":    ["entropia"],
            #         }
            #     print(f"[FIXED-CLF] {nombre_modelo}/{nombre_dataset}/{tipo} -> {clf_best}")
            # else:
            #     print(f"[WARN] Sin mapping para {nombre_modelo}/{nombre_dataset}/{tipo}. Se usa espacio original de clf__*")

            # Ajuste automático de n_iter para evitar el warning
            tam_espacio = space_size(param_distributions)
            if tam_espacio != float("inf"):
                n_iter_actual = min(n_iter_actual, max(1, tam_espacio))
                # opcional: log
                print(f"[RS] n_iter ajustado a {n_iter_actual} (espacio discreto = {tam_espacio})")


            t0 = time.perf_counter()
            search = RandomizedSearchCV(
                estimator=pipe,
                param_distributions=param_distributions,
                n_iter=n_iter_actual,
                scoring=scoring,
                refit="f1_macro",
                cv=cv_actual,
                n_jobs=n_jobs_actual,
                random_state=42,
                verbose=1
            )
            search.fit(X_train, y_train)
            elapsed = round(time.perf_counter() - t0, 3)

            # Hiperparámetros elegidos del sampler (fold-safe)
            best_params = search.best_params_
            dens = best_params.get("sampler__percentil_densidad")
            ries = best_params.get("sampler__percentil_riesgo") or best_params.get("sampler__percentil_dist")
            pur  = best_params.get("sampler__criterio_pureza")

            print(f"🧪 Mejor sampler → densidad={dens}, riesgo={ries}, pureza={pur}")


            # Best params (incluye sampler)
            best_params = search.best_params_
            dens = best_params.get("sampler__percentil_densidad")
            ries = best_params.get("sampler__percentil_riesgo")
            pur  = best_params.get("sampler__criterio_pureza")

            # CV (mejor índice)
            bi = search.best_index_
            cv_f1    = float(search.cv_results_['mean_test_f1_macro'][bi])
            cv_bacc  = float(search.cv_results_['mean_test_balanced_accuracy'][bi])
            cv_mcc   = float(search.cv_results_['mean_test_mcc'][bi])
            cv_kappa = float(search.cv_results_['mean_test_cohen_kappa'][bi])

            # Test
            best_est = search.best_estimator_
            y_pred   = best_est.predict(X_test)
            test_f1   = float(f1_score(y_test, y_pred, average='macro'))
            test_bacc = float(balanced_accuracy_score(y_test, y_pred))
            test_mcc  = float(matthews_corrcoef(y_test, y_pred))
            test_kappa= float(cohen_kappa_score(y_test, y_pred))

            gap_f1 = cv_f1 - test_f1

            # Registrar
            resultados_por_modelo[nombre_modelo].append({
                'dataset': nombre_dataset, 'tipo': 'foldsafe', 'tecnica': 'pcsmote',
                'densidad': dens, 'riesgo': ries, 'pureza': pur,
                'n_train': int(n_samples), 'n_test': int(X_test.shape[0]),
                'n_features': int(n_features), 'es_grande': bool(es_grande),
                'cv_splits': cv_actual.get_n_splits(), 'n_iter': n_iter_actual,
                'modelo': nombre_modelo, 'mejor_configuracion': str(best_params),
                'cv_f1_macro': cv_f1, 'cv_balanced_accuracy': cv_bacc,
                'cv_mcc': cv_mcc, 'cv_cohen_kappa': cv_kappa,
                'test_f1_macro': test_f1, 'test_balanced_accuracy': test_bacc,
                'test_mcc': test_mcc, 'test_cohen_kappa': test_kappa,
                'gap_f1_macro': gap_f1,
                'search_time_sec': elapsed, 'n_jobs_search': n_jobs_actual
            })

            # Volcado incremental y marca como hecho
            out_path = os.path.join(dir_out, f"resultados_{nombre_modelo}.csv")
            pd.DataFrame(resultados_por_modelo[nombre_modelo]).to_csv(out_path, index=False)
            ya_hechos[nombre_modelo].add((nombre_dataset, 'foldsafe', 'pcsmote', str(dens), str(ries), str(pur)))

        except Exception as e:
            print(f"❌ Error al validar {nombre_modelo} en {archivo_train}: {e}")

        # Limpieza de memoria
        gc.collect()

# -----------------------------
# Persistencia final por modelo
# -----------------------------
for nombre_modelo, lista in resultados_por_modelo.items():
    out_path = os.path.join(dir_out, f"resultados_{nombre_modelo}.csv")
    pd.DataFrame(lista).to_csv(out_path, index=False)
    print(f"📁 Resultados guardados: {out_path}")



📂 (1/4) Par: glass_train.csv  (tipo: foldsafe)
🔎 Técnica: pcsmote | Dataset: glass
⚙️ Validando modelo con RandomizedSearchCV: LogisticRegression
Fitting 5 folds for each of 10 candidates, totalling 50 fits
🧪 Mejor sampler → densidad=75, riesgo=75, pureza=entropia
⚙️ Validando modelo con RandomizedSearchCV: RandomForest
Fitting 5 folds for each of 10 candidates, totalling 50 fits
🧪 Mejor sampler → densidad=50, riesgo=50, pureza=entropia
⚙️ Validando modelo con RandomizedSearchCV: SVM
Fitting 5 folds for each of 10 candidates, totalling 50 fits
🧪 Mejor sampler → densidad=25, riesgo=50, pureza=entropia

📂 (2/4) Par: heart_train.csv  (tipo: foldsafe)
🔎 Técnica: pcsmote | Dataset: heart
⚙️ Validando modelo con RandomizedSearchCV: LogisticRegression
Fitting 5 folds for each of 10 candidates, totalling 50 fits
🧪 Mejor sampler → densidad=75, riesgo=25, pureza=entropia
⚙️ Validando modelo con RandomizedSearchCV: RandomForest
Fitting 5 folds for each of 10 candidates, totalling 50 fits
🧪 Mejor